In [1]:
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
file_path = "myData2.parquet"
df = pd.read_parquet(file_path)

In [1]:
# df.head(20)

In [76]:
# df['state'].value_counts()

In [101]:
# df.info(max_cols=100)

In [78]:
# data.head(20)

In [4]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

2023-12-12 12:02:06.263316: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-12 12:02:06.311961: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-12 12:02:06.312002: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-12 12:02:06.312035: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-12 12:02:06.320967: I tensorflow/core/platform/cpu_feature_g

In [5]:
selected_features = ['timestamp_seconds', # lowers the accuracy 
                     'node_memory_Percpu_bytes', 
                     'node_context_switches_total', 
                     'surfsara_power_usage', 
                     'node_netstat_Tcp_InSegs', 
                     'node_netstat_Tcp_OutSegs', 
                     'node_network_transmit_packets_total-sum', 
                     'node_filesystem_size_bytes-sum', 
                     'node_filesystem_files-sum', 
                     'node_memory_MemFree_bytes', 
                     'node_netstat_Tcp_InErrs']


In [6]:
# Add a new column 'failed_jobs' representing the target variable
df['failed_jobs'] = (df['state'] == 'FAILED').astype(int)


In [7]:
# Extract relevant columns
df_selected = df[['timestamp', 'state'] + selected_features].copy()

# Encode the target variable 'state' to binary (0 for "COMPLETED", 1 otherwise)
df_selected['target'] = (df_selected['state'] != 'COMPLETED').astype(int)

# Drop the original 'state' column
df_selected.drop('state', axis=1, inplace=True)

In [8]:
# Define time intervals
time_intervals = {'minute': '1T', 'hour': '1H', 'day': '1D'}


In [9]:
# Normalize selected features
scaler = MinMaxScaler()
df_selected[selected_features] = scaler.fit_transform(df_selected[selected_features])


In [10]:
# Function to prepare data for LSTM
def prepare_lstm_data(data, time_interval):
    data.set_index('timestamp', inplace=True) # FixMe
    data_resampled = data.resample(time_interval).sum()
    data_resampled['target'] = data_resampled['target'].clip(upper=1)  # Clip values to 1
    return data_resampled


In [11]:
# Function to create sequences for LSTM
def create_lstm_sequences(data, sequence_length):
    sequences, targets = [], []
    for i in range(len(data) - sequence_length):
        seq = data.iloc[i:i+sequence_length].values
        target = data.iloc[i+sequence_length]['target']
        sequences.append(seq)
        targets.append(target)
    return np.array(sequences), np.array(targets)

In [12]:
# Set sequence length
sequence_length = 30

In [13]:
# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


# 30 hours -> 7 hours

In [14]:
# Prepare data for LSTM with hourly intervals
lstm_data_hour = prepare_lstm_data(df_selected, time_intervals['hour'])

In [15]:
# Create sequences and targets
sequences_hour, targets_hour = create_lstm_sequences(lstm_data_hour, sequence_length)

# Split the data into training and testing sets
X_train_hour, X_test_hour, y_train_hour, y_test_hour = train_test_split(sequences_hour, targets_hour, test_size=0.3, random_state=42)


In [16]:
# Build the LSTM model
lstm_model_hour = Sequential()
lstm_model_hour.add(LSTM(50, input_shape=(X_train_hour.shape[1], X_train_hour.shape[2])))
lstm_model_hour.add(Dense(1, activation='sigmoid'))
lstm_model_hour.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])


In [17]:
# Train the model
lstm_model_hour.fit(X_train_hour, y_train_hour, epochs=20, batch_size=32, validation_split=0.1, callbacks=[early_stopping])


Epoch 1/20
54/54 [==============================] - 4s 32ms/step - loss: 0.4064 - mean_absolute_error: 0.4064 - val_loss: 0.3533 - val_mean_absolute_error: 0.3533
Epoch 2/20
54/54 [==============================] - 1s 23ms/step - loss: 0.3416 - mean_absolute_error: 0.3416 - val_loss: 0.3033 - val_mean_absolute_error: 0.3033
Epoch 3/20
54/54 [==============================] - 1s 22ms/step - loss: 0.2897 - mean_absolute_error: 0.2897 - val_loss: 0.2593 - val_mean_absolute_error: 0.2593
Epoch 4/20
54/54 [==============================] - 1s 21ms/step - loss: 0.2532 - mean_absolute_error: 0.2532 - val_loss: 0.2164 - val_mean_absolute_error: 0.2164
Epoch 5/20
54/54 [==============================] - 1s 22ms/step - loss: 0.2240 - mean_absolute_error: 0.2240 - val_loss: 0.1915 - val_mean_absolute_error: 0.1915
Epoch 6/20
54/54 [==============================] - 1s 22ms/step - loss: 0.2040 - mean_absolute_error: 0.2040 - val_loss: 0.1752 - val_mean_absolute_error: 0.1752
Epoch 7/20
54/54 [====

In [18]:
# Evaluate the model using Mean Absolute Error
mae = lstm_model_hour.evaluate(X_test_hour, y_test_hour, verbose=0)[1]
print(f'Model Mean Absolute Error: {mae:.4f}')


Model Mean Absolute Error: 0.1012


In [19]:
# # Evaluate the model
# loss, accuracy = lstm_model_hour.evaluate(X_test_hour, y_test_hour)
# print(f'Model Accuracy: {accuracy * 100:.2f}%')


In [20]:
# Function to make predictions on new data
def predict_future_failures(model, input_data, sequence_length, prediction_steps):
    predictions = []

    for _ in range(prediction_steps):
        # Make a prediction for the next time step
        prediction = model.predict(input_data.reshape(1, sequence_length, input_data.shape[1]))
        predictions.append(prediction[0, 0])

        # Shift the input data by one time step and append the new prediction
        input_data = np.roll(input_data, shift=-1, axis=0)
        input_data[-1, -1] = prediction[0, 0]

    return predictions


In [21]:
input_data = X_test_hour[160]  # Can be any valid starting point

# Number of time steps to predict into the future
prediction_steps = 7

# Make predictions
predicted_failures = predict_future_failures(lstm_model_hour, input_data, sequence_length, prediction_steps)

# Denormalize the predicted failures 
predicted_failures_denormalized = predicted_failures * (lstm_data_hour['target'].max() - lstm_data_hour['target'].min()) + lstm_data_hour['target'].min()

# Print the predicted failures
print("Predicted Failures for the Next 7 Hours:")
print(predicted_failures_denormalized)

# Evaluate the predictions using Mean Absolute Error
mae_predictions = np.mean(np.abs(predicted_failures - y_test_hour[160:160+prediction_steps]))
print(f'Mean Absolute Error for Predictions: {mae_predictions:.4f}')


1/1 [==============================] - 0s 20ms/step
Predicted Failures for the Next 7 Hours:
[0.99896365 0.9987356  0.99866486 0.99865806 0.998658   0.99863833
 0.99768925]
Mean Absolute Error for Predictions: 0.4286


# 30 minutes -> 7 minutes - Overfitting

In [108]:
# Prepare data for LSTM with minute intervals
lstm_data_minute = prepare_lstm_data(df_selected, time_intervals['minute'])

In [109]:
# Create sequences and targets
sequences_minute, targets_minute = create_lstm_sequences(lstm_data_minute, sequence_length)

# Split the data into training and testing sets
X_train_minute, X_test_minute, y_train_minute, y_test_minute = train_test_split(sequences_minute, targets_minute, test_size=0.3, random_state=42)


In [125]:
# Build the LSTM model
lstm_model_minute = Sequential()
# 50 -> 20
lstm_model_minute.add(LSTM(20, input_shape=(X_train_minute.shape[1], X_train_minute.shape[2])))
lstm_model_minute.add(Dropout(0.2)) # FixMe
lstm_model_minute.add(Dense(1, activation='sigmoid'))
lstm_model_minute.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [126]:
# Train the model
lstm_model_minute.fit(X_train_minute, y_train_minute, epochs=5, batch_size=32, validation_split=0.1, callbacks=[early_stopping])


Epoch 1/5
3258/3258 [==============================] - 62s 18ms/step - loss: 0.0337 - accuracy: 0.9915 - val_loss: 0.0112 - val_accuracy: 0.9977
Epoch 2/5
3258/3258 [==============================] - 59s 18ms/step - loss: 0.0110 - accuracy: 0.9980 - val_loss: 0.0090 - val_accuracy: 0.9984
Epoch 3/5
3258/3258 [==============================] - 59s 18ms/step - loss: 0.0098 - accuracy: 0.9984 - val_loss: 0.0091 - val_accuracy: 0.9984
Epoch 4/5
3258/3258 [==============================] - 59s 18ms/step - loss: 0.0096 - accuracy: 0.9985 - val_loss: 0.0086 - val_accuracy: 0.9986
Epoch 5/5
3258/3258 [==============================] - 60s 18ms/step - loss: 0.0094 - accuracy: 0.9985 - val_loss: 0.0088 - val_accuracy: 0.9985


In [127]:
# Evaluate the model
loss, accuracy = lstm_model_minute.evaluate(X_test_minute, y_test_minute)
print(f'Model accuracy (minute): {accuracy * 100:.2f}%')


1552/1552 [==============================] - 10s 6ms/step - loss: 0.0079 - accuracy: 0.9987
Model accuracy (minute): 99.87%


In [124]:
input_data = X_test_minute[200]  # Can be any valid starting point

# Number of time steps to predict into the future
prediction_steps = 7

# Make predictions
predicted_failures = predict_future_failures(lstm_model_minute, input_data, sequence_length, prediction_steps)

# Denormalize the predicted failures
predicted_failures_denormalized = predicted_failures * (lstm_data_minute['target'].max() - lstm_data_minute['target'].min()) + lstm_data_minute['target'].min()

# Print the predicted failures
print("Predicted failures for the next 7 minutes:")
print(predicted_failures_denormalized)


1/1 [==============================] - 0s 22ms/step
Predicted failures for the next 7 minutes:
[0.00059654 0.00060795 0.00062316 0.00061783 0.0006206  0.00061952
 0.00062073]


# 30 days -> 7 days

In [117]:
# Prepare data for LSTM with daily intervals
lstm_data_day = prepare_lstm_data(df_selected, time_intervals['day'])

In [118]:
len(lstm_data_day)

116

In [119]:
# Create sequences and targets
sequences_day, targets_day = create_lstm_sequences(lstm_data_day, sequence_length)

# Split the data into training and testing sets
X_train_day, X_test_day, y_train_day, y_test_day = train_test_split(sequences_day, targets_day, test_size=0.3, random_state=42)

# Build the LSTM model
lstm_model_day = Sequential()
lstm_model_day.add(LSTM(50, input_shape=(X_train_day.shape[1], X_train_day.shape[2])))
lstm_model_day.add(Dense(1, activation='sigmoid'))
lstm_model_day.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
lstm_model_day.fit(X_train_day, y_train_day, epochs=20, batch_size=32, validation_split=0.1, callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = lstm_model_day.evaluate(X_test_day, y_test_day)
print(f'Model accuracy (day): {accuracy * 100:.2f}%')

input_data = X_test_day[0]  # Can be any valid starting point

# Number of time steps to predict into the future
prediction_steps = 7

# Make predictions
predicted_failures = predict_future_failures(lstm_model_day, input_data, sequence_length, prediction_steps)

# Denormalize the predicted failures 
predicted_failures_denormalized = predicted_failures * (lstm_data_day['target'].max() - lstm_data_day['target'].min()) + lstm_data_day['target'].min()

# Print the predicted failures
print("Predicted failures for the next 7 days:")
print(predicted_failures_denormalized)


Epoch 1/20
2/2 [==============================] - 3s 523ms/step - loss: 0.6139 - accuracy: 0.7037 - val_loss: 0.3740 - val_accuracy: 1.0000
Epoch 2/20
2/2 [==============================] - 0s 71ms/step - loss: 0.6136 - accuracy: 0.7037 - val_loss: 0.3704 - val_accuracy: 1.0000
Epoch 3/20
2/2 [==============================] - 0s 64ms/step - loss: 0.6126 - accuracy: 0.7037 - val_loss: 0.3710 - val_accuracy: 1.0000
Epoch 4/20
2/2 [==============================] - 0s 66ms/step - loss: 0.6120 - accuracy: 0.7037 - val_loss: 0.3697 - val_accuracy: 1.0000
Epoch 5/20
2/2 [==============================] - 0s 65ms/step - loss: 0.6113 - accuracy: 0.7037 - val_loss: 0.3694 - val_accuracy: 1.0000
Epoch 6/20
2/2 [==============================] - 0s 66ms/step - loss: 0.6108 - accuracy: 0.7037 - val_loss: 0.3691 - val_accuracy: 1.0000
Epoch 7/20
2/2 [==============================] - 0s 64ms/step - loss: 0.6103 - accuracy: 0.7037 - val_loss: 0.3680 - val_accuracy: 1.0000
Epoch 8/20
2/2 [==========